# Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import opt_einsum as oe
from scipy.optimize import minimize, differential_evolution, NonlinearConstraint

from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense

from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme as int_slow
from quant_rotor.core.dense.de_solve_one_thermal_dense import integration_scheme as int_fast
import quant_rotor.models.dense.thermofield_boltz_funcs as bz

from quant_rotor.core.dense.t_amplitudes_periodic import t_periodic

from quant_rotor.models.dense.density_matrix import density_matrix_1

In [3]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(threshold=np.inf)

In [4]:
site = 3
state = 3
g = 0.9

# TF Hamimltonian

In [5]:
H, K, V = hamiltonian_general_dense(state, site, g)

## Check ocupations

In [6]:
eig_val_ED, eig_vec_ED = np.linalg.eigh(H)

In [7]:
eig_val_ED

array([-1.83417, -1.77634,  0.49623,  0.49623,  0.869  ,  0.869  ,  1.1542 ,  1.398  ,  1.89511,  1.89511,  1.93651,  2.2752 ,  2.2752 ,  2.9834 ,  3.09445,  3.10851,  3.11713,  3.23051,  3.23051,  3.27898,  3.27898,  3.5124 ,  3.5124 ,  3.55355,  3.55355,  3.83923,  3.90596])

In [8]:
index_ED = np.argsort(eig_val_ED)
psi_vec = eig_vec_ED[:, index_ED[0]]

D_ED = density_matrix_1(state, site, psi_vec, 0)

In [9]:
eig_val_ED[index_ED[0]]

np.float64(-1.8341740563709745)

In [10]:
eig_val_D_ED, eig_vec_D_ED  = np.linalg.eigh(D_ED)
index_ED = np.argsort(-eig_val_D_ED)

In [11]:
eig_val_D_ED[index_ED]

array([0.6274 , 0.36825, 0.00435])

##

In [12]:
state_2 = state**2

In [13]:
U, _ = bz.thermofield_change_of_basis(K)

K_tilda = bz.H_tilde_maker(K)

V_tensor = V.reshape(state, state, state, state)

I = np.eye(state)
V_prim = oe.contract('pqrs,mw,nv->pmqnrwsv', V_tensor, I, I,  optimize='optimal')

V_grouped = V_prim.reshape(state**2, state**2, state**2, state**2)

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U.T, U.T, V_grouped, U, U, optimize='optimal').reshape(state_2**2, state_2**2)

In [14]:
U, _ = bz.thermofield_change_of_basis(K)
I = np.eye(state)

K_prim = oe.contract('pq,mw->pmqw', K, I,  optimize='optimal')
K_grouped = K_prim.reshape(state_2, state_2)

V_tensor = V.reshape(state, state, state, state)
V_prim = oe.contract('pqrs,mw,nv->pnqmrvsw', V_tensor, I, I,  optimize='optimal')
V_grouped = V_prim.reshape(state_2**2, state_2**2)

In [15]:
H_TF, _, _ = hamiltonian_dense(state_2, site, g, K_import=K_tilda, V_import=V_tilda, Import=True)
# H_TF, _, _ = hamiltonian_dense(state_2, site, g, K_import=K_grouped, V_import=V_grouped, Import=True)

# Degenerate ground state basis.

In [16]:
eig_val_TF, eig_vec_TF = np.linalg.eigh(H_TF)

In [17]:
index_TF = np.argsort(-eig_val_TF)
psi_vec = eig_vec_TF[index_TF[0]]

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [18]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [19]:
deg_ground_basis = eig_vec_TF[:, index_TF[:27]]

In [20]:
deg_state_ocup = np.zeros((9, 27))

for i in range(state**2*site):

    rho_site_0 = density_matrix_1(state_2, site, deg_ground_basis[:, i], 0)

    eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)

    index = np.argsort(-eig_val_D)
    deg_state_ocup[:, i] = eig_val_D[index]

In [21]:
mask = np.argsort(-deg_state_ocup[0, :])

In [22]:
print(np.array2string(deg_state_ocup[:, mask], precision=3))

[[0.363 0.346 0.333 0.33  0.313 0.31  0.308 0.306 0.304 0.301 0.29  0.276 0.265 0.245 0.243 0.241 0.238 0.236 0.235 0.233 0.231 0.226 0.225 0.219 0.218 0.205 0.197]
 [0.29  0.277 0.266 0.264 0.251 0.248 0.246 0.245 0.243 0.241 0.232 0.221 0.212 0.196 0.194 0.193 0.19  0.189 0.198 0.186 0.184 0.181 0.18  0.175 0.174 0.164 0.158]
 [0.164 0.156 0.151 0.149 0.142 0.14  0.139 0.139 0.137 0.136 0.131 0.125 0.12  0.194 0.184 0.168 0.132 0.144 0.188 0.113 0.167 0.162 0.147 0.144 0.173 0.16  0.158]
 [0.06  0.069 0.089 0.077 0.118 0.106 0.091 0.087 0.123 0.083 0.126 0.094 0.095 0.155 0.147 0.135 0.107 0.115 0.158 0.105 0.133 0.129 0.118 0.115 0.138 0.128 0.126]
 [0.048 0.055 0.071 0.061 0.094 0.084 0.073 0.069 0.098 0.067 0.1   0.075 0.084 0.111 0.11  0.109 0.106 0.107 0.106 0.098 0.104 0.102 0.102 0.099 0.098 0.093 0.089]
 [0.027 0.031 0.04  0.038 0.053 0.048 0.045 0.051 0.055 0.06  0.057 0.074 0.076 0.088 0.083 0.076 0.074 0.065 0.089 0.091 0.075 0.073 0.072 0.081 0.078 0.08  0.089]
 [0.021 0.

In [23]:
deg_state_d_2 = np.zeros((27))

for i in range(state**2*site):

    rho_site_0 = density_matrix_1(state_2, site, deg_ground_basis[:, i], 0)

    deg_state_d_2[i] = np.trace(rho_site_0**2)

/var/folders/vt/f4sbj2d11gbbb9q5816khz2m0000gn/T/ipykernel_21174/2723689836.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  deg_state_d_2[i] = np.trace(rho_site_0**2)


In [24]:
mask_d2 = np.argsort(-deg_state_d_2)

In [25]:
deg_state_d_2[mask_d2]

array([0.2326 , 0.21223, 0.19414, 0.16932, 0.16759, 0.16577, 0.15697, 0.14944, 0.14688, 0.14577, 0.14482, 0.14399, 0.14271, 0.14149, 0.14133, 0.13594, 0.13496, 0.1346 , 0.13319, 0.13202, 0.12959, 0.12692, 0.12623, 0.12436, 0.12317, 0.12144, 0.11959])

# Optimal ground state basis.

In [146]:
def f_max(C, ground_NO):
    # C is variable; ground_NO is fixed from args
    psi = ground_NO @ C
    D = density_matrix_1(state*2, site, psi, 0)
    return float(np.real(np.trace(D**2)))

def constr_ineq(C, ground_NO):
    return -f_max(C, ground_NO)

# optional equality constraint: <psi|psi> = 1
def constr_eq(C):
    return float(C @ C) - 1.0

# initial guess
C0 = np.full((state**2*site), -1 / (state**2*site))
best = 0

# for i in range(100):
#     # uniform random numbers between -0.5 and 0.5
#     C0 = np.random.uniform(-1, 1, size=(state**2*site))
#     C0 /= np.linalg.norm(C0)

#     print(C0)

res = minimize(
    constr_ineq,
    x0=C0,
    args=(deg_ground_basis,),
    method='SLSQP',
    constraints=[
        {'type': 'eq', 'fun': constr_eq}
    ],
    options={'ftol': 1e-6, 'maxiter': 1000}
)

    # C_opt = res.x

    # val = f_max(C_opt, deg_ground_basis)
    # # print(f"Trial {i+1:2d}: Tr(D^2) = {val:.6f}")
    # best = max(best, val)
    # print(i, best)

In [137]:
# def f_max(C, ground_NO):
#     # C is variable; ground_NO is fixed from args
#     psi = ground_NO @ C
#     D = density_matrix_1(state*2, site, psi, 0)
#     # purity is real; keep scalar return
#     return float(np.real(np.trace(D @ D)))  # or np.trace(D**2)

# # objective: minimize 1 - f_max  <=>  maximize f_max
# def obj(C, ground_NO):
#     return f_max(C, ground_NO) - 1.0

# # equality: ||C||^2 = 1
# def constr_eq(C):
#     return float(C @ C) - 1.0

# # inequality: f_max(C, ground_NO) >= ε
# epsilon = 1e-12  # set to 0 if non-strict is fine
# def constr_pos(C, ground_NO):
#     return f_max(C, ground_NO) - epsilon

# # initial guess (1D vector)
# C0 = np.full(state**2 * site, 1.0 / (state**2 * site), dtype=float)

# res = minimize(
#     obj,
#     x0=C0,
#     args=(deg_ground_basis,),   # pass ground_NO to obj
#     method='SLSQP',
#     constraints=[
#         {'type': 'eq',   'fun': constr_eq},
#         {'type': 'ineq', 'fun': obj, 'args': (deg_ground_basis,)},
#         {'type': 'ineq', 'fun': constr_pos, 'args': (deg_ground_basis,)}
#     ],
#     options={'ftol': 1e-2, 'maxiter': 1000}
# )

In [138]:
# # --------- Your physics-specific pieces (edit/keep as-is) ---------
# def f_max(C, ground_NO):
#     """
#     Returns the scalar objective you want to maximize, e.g. Tr(D^2) (purity).
#     C:      (n,) real vector (variables)
#     ground_NO: operator/matrix used to build psi
#     """
#     psi = ground_NO @ C                     # shape check: (dim_state,)
#     D = density_matrix_1(state*2, site, psi, 0)
#     return float(np.real(np.trace(D @ D)))  # Tr(D^2)

# # equality constraint: ||C||^2 = 1
# def constr_eq(C):
#     return 1.0 - float(C @ C)

# # objective for local solver: minimize 1 - f_max  <=>  maximize f_max
# def obj(C, ground_NO):
#     return 1.0 - f_max(C, ground_NO)

# # --------- Global stage (Differential Evolution) -------------------
# def obj_DE(y, ground_NO):
#     """
#     DE doesn't handle equality constraints directly.
#     We enforce ||C||=1 by normalizing y -> C = y / ||y|| inside the objective.
#     """
#     norm = np.linalg.norm(y)
#     if norm == 0.0:
#         return 1.0  # worst-case if y=0 (objective = 1 - f_max), push away from 0
#     C = y / norm
#     return obj(C, ground_NO)

# def run_global_then_local(ground_NO, state, site, seed=123, de_maxiter=300, slsqp_maxiter=2000):
#     n = state**2 * site  # dimension of C

#     # --- Differential Evolution: rough global search on the unit sphere via normalization ---
#     bounds = [(-1.0, 1.0)] * n  # coarse box; normalization inside obj_DE puts us on the sphere
#     de_res = differential_evolution(
#         obj_DE,
#         bounds=bounds,
#         args=(ground_NO,),
#         strategy='best1bin',
#         popsize=15,
#         maxiter=de_maxiter,
#         tol=0.01,        # loosen if too slow; tighten if needed
#         mutation=(0.5, 1.0),
#         recombination=0.7,
#         polish=False,    # we'll do the polishing ourselves with SLSQP
#         seed=seed
#     )

#     # Build a feasible starting point for the local solver (unit norm)
#     y_best = de_res.x
#     C0 = y_best / max(np.linalg.norm(y_best), 1e-16)

#     # --- Local refinement with SLSQP under the equality constraint ||C||=1 ---
#     slsqp_res = minimize(
#         obj,
#         x0=C0,
#         args=(ground_NO,),
#         method='SLSQP',
#         constraints=[{'type': 'eq', 'fun': constr_eq}],
#         options={'ftol': 1e-6, 'maxiter': slsqp_maxiter, 'disp': False}
#     )

#     # Evaluate f_max at both stages
#     fmax_de   = f_max(C0, ground_NO)                  # after normalization of DE result
#     fmax_local = f_max(slsqp_res.x, ground_NO) if slsqp_res.success else None

#     # Simple summary
#     summary = {
#         'DE': {
#             'fun': 1.0 - fmax_de,
#             'f_max': fmax_de,
#             'nfev': de_res.nfev,
#             'message': de_res.message,
#             'success': de_res.success
#         },
#         'SLSQP': {
#             'fun': slsqp_res.fun,
#             'f_max': fmax_local,
#             'nit': slsqp_res.nit,
#             'nfev': slsqp_res.nfev,
#             'message': slsqp_res.message,
#             'success': slsqp_res.success
#         },
#         'C_global_start': C0,
#         'C_opt': slsqp_res.x if slsqp_res.success else C0
#     }
#     return summary

In [139]:
# summary = run_global_then_local(deg_ground_basis, state=state, site=site)
# print("Global (DE)  -> success:", summary['DE']['success'],  "f_max:", summary['DE']['f_max'])
# print("Local (SLSQP)-> success:", summary['SLSQP']['success'], "f_max:", summary['SLSQP']['f_max'])

In [140]:
# def obj_penalty(C, ground_NO, lam=1e3):
#     val = f_max(C, ground_NO)
#     return (1.0 - val) + lam * (max(0, 1.0 - val))**2

# norm_con = NonlinearConstraint(lambda C: float(C @ C), 1.0, 1.0)

# res = minimize(
#     obj_penalty, C0,
#     args=(deg_ground_basis, 1e3),      # increase lambda if needed
#     method='trust-constr',
#     constraints=[norm_con],
#     options={'verbose': 0, 'gtol': 1e-10, 'xtol': 1e-12, 'barrier_tol': 1e-12}
# )

In [147]:
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.0439826809025278
       x: [ 4.063e-02 -3.736e-02 ... -5.860e-01 -1.004e-01]
     nit: 32
     jac: [-7.186e-03  6.551e-03 ...  1.031e-01  1.765e-02]
    nfev: 902
    njev: 32

In [142]:
# C_opt = summary['C_opt']

C_opt = res.x

In [135]:
C_opt @ C_opt

np.float64(1.0000005922236501)

In [112]:
C_opt

array([-0.04063,  0.03736, -0.03722,  0.13279,  0.0332 , -0.08272, -0.13517, -0.06367,  0.0818 ,  0.17391,  0.05423, -0.02024,  0.0255 ,  0.38366,  0.26913,  0.02091,  0.33415,  0.01299,  0.01103, -0.06293, -0.04568, -0.03167, -0.08881,  0.44419,  0.09488,  0.58601,  0.10036])

In [113]:
psi = deg_ground_basis @ C_opt

In [114]:
D_1 = density_matrix_1(state**2, site, psi, 0)

In [115]:
eig_val_opt, opt_grd_basis = np.linalg.eigh(D_1)
index = np.argsort(-eig_val_opt)

In [116]:
constr_ineq(C_opt, deg_ground_basis)

0.9560173185913747

In [117]:
-0.9466585218254122

-0.9466585218254122

In [106]:
eig_val_opt[index]

array([0.44386, 0.35494, 0.20066, 0.00024, 0.00019, 0.00011, 0.     , 0.     , 0.     ])

In [107]:
opt_grd_basis

array([[ 0.57712+0.j, -0.01638+0.j,  0.00005+0.j, -0.01638+0.j,  0.57712-0.j, -0.00042+0.j,  0.00004+0.j,  0.00042+0.j, -0.57735-0.j],
       [-0.     +0.j,  0.02007+0.j,  0.00007+0.j, -0.     +0.j, -0.70682+0.j, -0.00051+0.j, -0.     +0.j, -0.00052+0.j, -0.70711+0.j],
       [-0.81617+0.j, -0.01158+0.j,  0.00004+0.j,  0.02317+0.j,  0.40808+0.j, -0.0003 +0.j, -0.00006+0.j,  0.0003 +0.j, -0.40825+0.j],
       [ 0.02838+0.j, -0.     +0.j, -0.     +0.j,  0.9996 +0.j, -0.     +0.j,  0.     +0.j,  0.00073+0.j,  0.     +0.j, -0.     +0.j],
       [ 0.00009+0.j,  0.     +0.j, -0.     +0.j,  0.00073+0.j, -0.     +0.j, -0.     +0.j, -1.     +0.j,  0.     +0.j,  0.     +0.j],
       [-0.     +0.j, -0.9996 +0.j, -0.     +0.j,  0.     +0.j, -0.02838+0.j,  0.     +0.j, -0.     +0.j,  0.00007+0.j,  0.     +0.j],
       [ 0.     +0.j, -0.00009+0.j, -0.     +0.j,  0.     +0.j,  0.00073+0.j, -0.     +0.j, -0.     +0.j, -1.     +0.j, -0.     +0.j],
       [ 0.     +0.j, -0.     +0.j,  0.9996 +0.j,  0.  

## Check Optimization

In [42]:
n = state**2 * site
best = 0

for i in range(100):
    # uniform random numbers between -0.5 and 0.5
    C0 = np.random.uniform(-1/(state**2*site), 1/(state**2*site), size=n)

    # normalize to unit length (important for ||C||=1 constraint)
    C0 /= np.linalg.norm(C0)

    val = f_max(C0, deg_ground_basis)
    # print(f"Trial {i+1:2d}: Tr(D^2) = {val:.6f}")
    best = max(best, val)

print("Best random purity found:", best)

Best random purity found: 0.027873691407678498


In [43]:
C_lambda = deg_ground_basis - np.outer(psi, psi @ deg_ground_basis)

In [44]:
C_lambda.shape

(729, 27)

In [45]:
S = C_lambda @ C_lambda.T

In [46]:
eig_val_S, eig_vec_S = np.linalg.eigh(S)

In [47]:
eig_vec_S.shape

(729, 729)

In [48]:
index_non_zero = np.where(np.abs(eig_val_S) > 1e-5)[0]

In [49]:
eig_val_S[index_non_zero]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [50]:
C_new = eig_vec_S[:, index_non_zero] * eig_val_S[index_non_zero]**(-0.5)

In [51]:
C_new.shape

(729, 26)

In [52]:
np.diag(C_new.T @ S @ C_new)

array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])

In [53]:
C_new[:, 1].shape

(729,)

In [54]:
def d_squared_overlap_basis(theta: float):
    deg_state_d_2 = np.zeros((27))

    for i in range(state**2*site - 1):

        psi_i = np.cos(theta) * psi + np.sin(theta) * C_new[:, i]

        rho_site_0 = density_matrix_1(state_2, site, psi_i, 0)

        deg_state_d_2[i] = np.trace(rho_site_0**2)

    return deg_state_d_2

In [55]:
print(-np.sort(-d_squared_overlap_basis(0)))

[0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.1196 0.    ]


/var/folders/vt/f4sbj2d11gbbb9q5816khz2m0000gn/T/ipykernel_21174/4112026237.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  deg_state_d_2[i] = np.trace(rho_site_0**2)


In [56]:
theta = 0.1

g_i = (d_squared_overlap_basis(theta) - d_squared_overlap_basis(-theta)) / 2 * np.abs(theta)

h_i = (2 * d_squared_overlap_basis(0) - d_squared_overlap_basis(theta) - d_squared_overlap_basis(-theta)) / theta**2

/var/folders/vt/f4sbj2d11gbbb9q5816khz2m0000gn/T/ipykernel_21174/4112026237.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  deg_state_d_2[i] = np.trace(rho_site_0**2)


In [57]:
g_i

array([ 0.00004, -0.00004, -0.00007,  0.00006,  0.00001, -0.00005,  0.00011, -0.00003, -0.00015, -0.00004, -0.00004,  0.00002, -0.00012,  0.00001,  0.00014, -0.00006,  0.00006, -0.00008,  0.00003,  0.00002, -0.00007, -0.00003,  0.00003,  0.00001, -0.00007, -0.00002,  0.     ])

In [58]:
h_i

array([-0.0339 , -0.01191, -0.05185, -0.07671,  0.00056, -0.10121, -0.06905, -0.0252 , -0.14608, -0.02865, -0.04388, -0.06426, -0.07878, -0.06028, -0.13241, -0.07028, -0.0672 , -0.04266, -0.02149, -0.00339, -0.02708, -0.06233, -0.01985, -0.01486, -0.06596, -0.01888,  0.     ])

In [59]:
np.divide(g_i, h_i)

/var/folders/vt/f4sbj2d11gbbb9q5816khz2m0000gn/T/ipykernel_21174/1969988016.py:1: RuntimeWarning: invalid value encountered in divide
  np.divide(g_i, h_i)


array([-0.0011 ,  0.00297,  0.00136, -0.00075,  0.0195 ,  0.00053, -0.00157,  0.0013 ,  0.00103,  0.00151,  0.00088, -0.00037,  0.00149, -0.00025, -0.00105,  0.00087, -0.00088,  0.00192, -0.0013 , -0.00485,  0.00263,  0.00051, -0.0013 , -0.00077,  0.00106,  0.00103,      nan])

# Producing TF in new basis.

In [60]:
K_opt_grd = opt_grd_basis.T.conj() @ K_tilda @ opt_grd_basis

matrix_p_to_NO_V = np.kron(opt_grd_basis, opt_grd_basis)

V_opt_grd = matrix_p_to_NO_V.conj().T @ V_tilda @ matrix_p_to_NO_V

# Canclulations

In [61]:
H_TF_opt, _, _ = hamiltonian_dense(state_2, site, 1, K_import=K_opt_grd, V_import=V_opt_grd, Import=True)

In [62]:
eig_val_TF_opt_ED, eig_vec_TF_opt_ED = np.linalg.eigh(H_TF_opt)
ground_TF_opt_ED = np.min(eig_val_TF_opt_ED)
ground_TF_opt_ED

np.float64(-1.8341740563709799)

In [63]:
index_TF = np.argmin(eig_val_TF_opt_ED)
psi_vec = eig_vec_TF[index_TF]

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [64]:
eig_va, eig_vc = np.linalg.eigh(rho_site_0)

eig_va

array([0.07406, 0.07407, 0.07407, 0.07407, 0.07407, 0.07409, 0.14815, 0.18519, 0.22222])

In [65]:
H_TF, _, _ = hamiltonian_dense(state_2, site, 1, K_import=K_tilda, V_import=V_tilda, Import=True)

In [66]:
eig_val_TF_ED, eig_vec_TF_ED = np.linalg.eigh(H_TF)
ground_TF_ED = np.min(eig_val_TF_ED)
ground_TF_ED

np.float64(-1.8341740563709767)

In [67]:
H, K_NO, V_NO = hamiltonian_general_dense(state, site, g)

In [68]:
eig_val_ED, eig_vec_ED = np.linalg.eigh(H)
ground_ED = np.min(eig_val_ED)
ground_ED

np.float64(-1.8341740563709745)

***Imag. Time propagation***

In [69]:
time_opt, energy_TD_CCC_opt, one_max_TDCC_opt, two_max_TDCC_opt = int_slow(site, state_2, 1, t_init=0, t_final=10, nof_points=10000, import_K_V_TF=True, K_import=K_opt_grd, V_import=V_opt_grd.reshape(state_2, state_2, state_2, state_2))

In [70]:
time_TF, energy_TD_TF_CCC, one_max_TD_TF_CC, two_max_TD_TF_CC = int_slow(site, state_2, 1, t_init=0, t_final=10, nof_points=10000, import_K_V_TF=True, K_import=K_tilda, V_import=V_tilda.reshape(state_2, state_2, state_2, state_2))

In [71]:
time, energy_TD_CCC, one_max_TDCC, two_max_TDCC = int_slow(site, state, 1, t_init=0, t_final=10, nof_points=10000, import_K_V_TF=True, K_import=K_NO, V_import=V_NO.reshape(state, state, state, state))

***CCC***

In [72]:
energy_CCC, t_1, t_2 = t_periodic(site, state_2, 1, K_import=K_opt_grd, V_import=V_opt_grd.reshape(state_2, state_2, state_2, state_2), Import=True)

[[ 0.13828+0.j -0.     +0.j -0.     +0.j  0.     +0.j -0.     +0.j -0.     +0.j -0.     +0.j -0.     +0.j  0.     +0.j]]


ValueError: Diverges.

In [ ]:
one_max, two_max, energy_CCC, t_1, t_2 = t_periodic(site, state_2, 1, K_import=K_tilda, V_import=V_tilda.reshape(state_2, state_2, state_2, state_2), Import=True)

In [ ]:
one_max, two_max, energy_CCC, t_1, t_2 = t_periodic(site, state, g, NO=True)

# Analisis

In [ ]:
plt.plot(time, -energy_TD_CCC, label=f"NO g = {g}")
plt.plot(time_opt, -energy_TD_CCC_opt, label=f"TF-Opt g = {g}")
plt.plot(time_TF, -energy_TD_TF_CCC, label=f"TF g = {g}")
plt.title("Energy Imaginary Time Propagation")
plt.xlabel("Time")
plt.ylabel("Energy")
plt.legend()
plt.show()

print("NO", energy_TD_CCC[-1])
print("TF-Opt", energy_TD_CCC_opt[-1])
print("TF", energy_TD_TF_CCC[-1])

In [ ]:
ground_TF_opt_ED

In [ ]:
energy_TD_CCC_opt[-1]

In [ ]:
energy_TD_CCC[-1]

In [ ]:
energy_CCC